In [1]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
    width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [2]:
import operator

from pyspark.sql import Row
from pyspark.sql.functions import udf, when, rand, struct, col, lit
from pyspark.sql.types import StringType, IntegerType, StructField, StructType, DoubleType, ArrayType

In [3]:
import visualize
from com.ophelia.OpheliaVendata import OpheliaVendata

In [4]:
ophelia = OpheliaVendata("Risk Classification Demo")


22:36:14.249 Ophelia [INFO] ¡Hullo! My name is Ophelia and I'm pleased to meet you
22:36:14.249 Ophelia [INFO] I am an artificial assistant for intelligent investment
22:36:14.249 Ophelia [INFO] Welcome to your asset allocation system
22:36:14.249 Ophelia [INFO] V for Vendata...

                         - By. Vendetta Gentleman Club -                     

           █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █
           █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ █
           █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █
           █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █
           █ ╬ ╬ ╬ █ █ █ █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ █ █ █ █ ╬ ╬ ╬ █
           █ ╬ ╬ █ █ ╬ ╬ ╬ ╬ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ ╬ ╬ ╬ ╬ █ █ ╬ ╬ █
           █ ╬ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ ╬ ╬ ╬ ╬ ╬ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ ╬ █
           █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █
           █ ╬ ╬ ╬ ╬ █ █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ 

In [ ]:
path = "data/raw/csv/bank.csv"
df = spark.read.csv(path, header=True)
df.show(5, False)

In [5]:
spark = ophelia.ophelia_session
path = "data/master/ophelia/data/OpheliaData/risk_classification/"
random_sample_df = spark.read.parquet(path)
random_sample_df.show(5, False)

+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------+
|age|job       |marital|education|gender|child|saving|insight|backup|struct                                                  |
+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------+
|59 |admin.    |married|secondary|male  |1    |0     |1      |1     |[male, 59, secondary, admin., married, 1, 0, 1, 1]      |
|56 |admin.    |married|secondary|female|0    |1     |0      |1     |[female, 56, secondary, admin., married, 0, 1, 0, 1]    |
|41 |technician|married|secondary|female|1    |1     |0      |0     |[female, 41, secondary, technician, married, 1, 1, 0, 0]|
|55 |services  |married|secondary|male  |1    |0     |0      |1     |[male, 55, secondary, services, married, 1, 0, 0, 1]    |
|54 |admin.    |married|tertiary |female|1    |0     |0      |1     |[female, 54, tertiary, admin., married, 1,

In [6]:
schema_tree = ArrayType(StructType([
    StructField("weight", DoubleType(), True),
    StructField("risk_label", StringType(), True),
    StructField("vote", IntegerType(), True)
]))

In [13]:
sc = ophelia.ophelia_spark.build_spark_context()
sc.addFile("../ophelia/", recursive = True)

22:36:56.096 Ophelia [INFO] OpheliaSpark Context Initialized Successfully
22:36:56.096 Ophelia [INFO] Build OpheliaSpark Session
22:36:56.101 Ophelia [INFO] OpheliaSpark Session Version: 2.4.5
22:36:56.101 Ophelia [WARN] Please, Be Aware To Set App Name Next Time...


In [14]:
tree_udf = udf(ophelia.ophelia_class.tree_generator, schema_tree)
classification_udf = udf(ophelia.ophelia_class.run_classification_risk, StringType())

In [15]:
result_df = random_sample_df.select("*", tree_udf(col("struct")).alias("tree")).cache()
result_df.show(5, False)

+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|age|job       |marital|education|gender|child|saving|insight|backup|struct                                                  |tree                                                                                                                                                   |
+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|59 |admin.    |married|secondary|male  |1    |0     |1      |1     |[male, 59, secondary, admin., married, 1, 0, 1, 1]      |[[0.0072, A, 1], [0.3205, MC, 7], [9.

In [16]:
final = result_df.select("age", "job", "marital", "education", "gender",
                         "child", "saving", "insight", "backup",
                         classification_udf(col("tree")).alias("risk_label"))

final.groupBy("risk_label").count().show(10, False)
final.show(10, False)

+----------+-----+
|risk_label|count|
+----------+-----+
|M         |360  |
|C         |628  |
|A         |3752 |
|MC        |3294 |
|MA        |3128 |
+----------+-----+

+---+----------+--------+---------+------+-----+------+-------+------+----------+
|age|job       |marital |education|gender|child|saving|insight|backup|risk_label|
+---+----------+--------+---------+------+-----+------+-------+------+----------+
|59 |admin.    |married |secondary|male  |1    |0     |1      |1     |MC        |
|56 |admin.    |married |secondary|female|0    |1     |0      |1     |MA        |
|41 |technician|married |secondary|female|1    |1     |0      |0     |MA        |
|55 |services  |married |secondary|male  |1    |0     |0      |1     |MA        |
|54 |admin.    |married |tertiary |female|1    |0     |0      |1     |MA        |
|42 |management|single  |tertiary |male  |0    |1     |1      |0     |A         |
|56 |management|married |tertiary |male  |1    |1     |1      |0     |MC        |
|60 |ret

In [ ]:
risk_classification_path = ophelia.ophelia_write.write_parquet(
    df=final,
    output_type="model",
    project="OpheliaRiskClassifier",
    part="risk_label"
)